In [1]:
import os
import time
import json
import pandas as pd
import csv

In [2]:
start_time = time.time()
aircraft = 'AD4804'
out_file = open('ads-b_AD4804.csv','a')
writer = csv.writer(out_file)
command = 'curl "https://data-live.flightradar24.com/zones/fcgi/feed.js?bounds=45,35,-78,-68&faa=1&mlat=1&flarm=1&adsb=1&gnd=1&air=1&vehicles=1&estimated=1&maxage=7200&gliders=1&stats=1" > temp.json'
curr_time = time.time()
total_time = curr_time - start_time

In [3]:
run_time_hours = 144
query_wait_time = 8

while total_time < (run_time_hours * 60 * 60):
    try:
        os.system(command)
        out = json.load(open("temp.json"))
        out = {key: out[key] for key in out.keys() if (key != "full_count" and key != "version" and key != "stats")}
        df = pd.DataFrame(out).T.reset_index(drop = False)
        df = df[df[0] == aircraft]
        
        for i in range(df.shape[0]):
            row = df.iloc[i]
            writer.writerow([curr_time] + row.tolist())
        
        curr_time = time.time()
        total_time = curr_time - start_time
        time.sleep(query_wait_time)
    except:
        continue

In [4]:
out_file.close()
os.system("rm temp.json")

0